# Predicting House Sale Prices

This notebook is a challenge from datquest.io

The objective is to build a model to predict house sale prices from the city of Ames, Iowa, United States
<br>
The dataset has data from 2006 to 2010

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
data = pd.read_csv('AmesHousing.tsv', delimiter='\t')

In [4]:
data.shape

(2930, 82)

In [5]:
data.head(10)

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900
5,6,527105030,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,6,2010,WD,Normal,195500
6,7,527127150,120,RL,41.0,4920,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,213500
7,8,527145080,120,RL,43.0,5005,Pave,NaN,IR1,HLS,...,0,NaN,NaN,NaN,0,1,2010,WD,Normal,191500
8,9,527146030,120,RL,39.0,5389,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,3,2010,WD,Normal,236500
9,10,527162130,60,RL,60.0,7500,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,6,2010,WD,Normal,189000


In [36]:
def select_features(df, min_corr, min_unique_cat):
    """
    Receive a df, a minimum correlation and a minimum unique categories
    Returns a df with the selected features
    """
    
    df = df.copy()
    
    #Selecting only numeric columns with a correlation with SalePrice higher than the 'min_corr' parameter
    numeric = df.select_dtypes(include = ['int64','float64']).columns
    
    correlations = df[numeric].corr().abs().sort_values(by='SalePrice')
    
    numeric = correlations[correlations['SalePrice'] > min_corr]
    numeric = numeric.index.tolist()
    
    #Filling null values in numeric columns with the mode
    for c in numeric:
        df[c] = df[c].fillna(df[c].mode().iloc[0])
        
    #Dropping null values from other columns
    df = df.dropna(axis = 1)
    
    #Selecting non numeric columns and transforming them into dummy columns
    non_numeric = list(df.select_dtypes(include = ['object']).columns)

    for col in non_numeric:
        df[col] = df[col].astype('category')
    
    #Selecting only columns with a number of unique categories higher than the 'min_unique_cat' parameter
    for col in non_numeric:
        if len(df[col].value_counts().index) < min_unique_cat:
            df = df.drop(col, axis = 1)
            
    #Creating a new df containg the numeric columns and the dummy columns
    df = pd.concat([
        df[numeric],
        pd.get_dummies(df.select_dtypes(include=['category']))
        ],
        axis=1
    )
    
    return df

In [32]:
def train_and_test(df, seed=4132):
    """
    Receives a df and a seed for random sample
    Returns the RMSE value based on the predictions done by a Linear Regression model
    
    """
    
    #Select all columns except for SalePrice which is the target
    features = df.columns.to_list()
    features.remove('SalePrice')
    
    #Separing randomly into train and test datasets
    np.random.seed(seed)
    sample = np.random.rand(len(df)) < 0.6

    train = df[sample]
    test = df[~sample]

    #Training and predicting with a Linear Regression model
    lr = LinearRegression()
    lr.fit(train[features], train['SalePrice'])

    predictions = lr.predict(test[features])
    
    #Calculating the RMSE of the model
    rmse = np.sqrt(mean_squared_error(test['SalePrice'], predictions))
    return rmse

## Data Cleaning

Dropping  columns that are not useful for the model

In [8]:
data = data.drop(['Order', 'PID'], axis = 1)

Dropping  columns with more thann 5% rows with null values

In [9]:
missing = data.isnull().sum()
missing_columns = missing[missing > len(data) * 0.05]
data  = data.drop(missing_columns.index, axis=1)

Testing the model with multiple minimum variance and minimum unique categories

In [29]:
min_var = np.arange(0.3,0.8,0.1)
min_unique = np.arange(5,11,1)
rmses = []
for v in min_var:
    for u in min_unique:
        df = select_features(data, v, u)
        rmses.append([v,u,train_and_test(df)])
        
rmses = pd.DataFrame(rmses)
rmses.columns = ['min_var', 'min_unique_cat', 'rmse']

Getting the lowest RMSE values

In [31]:
rmses.sort_values(by='rmse')

,min_var,min_unique_cat,rmse
6,0.4,5,29651.094952
0,0.3,5,29745.178551
12,0.5,5,30771.491739
10,0.4,9,31065.866732
4,0.3,9,31091.814255
11,0.4,10,31103.379460
5,0.3,10,31132.879694
18,0.6,5,31186.465852
24,0.7,5,31316.170179
9,0.4,8,31783.703272
